In [1]:
import os
print(os.getcwd())
os.chdir('d:\\vscode_machineLearning\\internship\\Customer-Churn-Prediction')
print(os.getcwd())

d:\vscode_machineLearning\internship\Customer-Churn-Prediction\experiments\model_training
d:\vscode_machineLearning\internship\Customer-Churn-Prediction


In [2]:
import pandas as pd

In [3]:
train_df = pd.read_csv(r'artifacts/transformed_data/train_transformed.csv')
test_df = pd.read_csv(r'artifacts/transformed_data/test_transformed.csv')

## Entity

In [4]:
from dataclasses import dataclass
from pathlib import Path

In [5]:
@dataclass(frozen=True)
class ModelTrainerConfig:
    train_data : Path
    test_data : Path
    model_dir : Path
    model_ojb : str
    n_estimators : int
    oob_score : bool

## configuration

In [6]:
from churnPredictor.constants import *
from churnPredictor.utils import *

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_file_path = CONFIG_FILE_PATH,
        schema_file_path = SCHEMA_FILE_PATH,
        params_file_path=PARAMS_FILE_PATH):

        self.config = read_yaml(config_file_path)
        self.schema = read_yaml(schema_file_path)
        self.params = read_yaml(params_file_path)

        create_dirs([self.config.artifacts_root])
        
    
    def get_modelTrainer_config(self):
        config = self.config.model_trainer
        params = self.params.RandomForest

        create_dirs([config.model_dir])
        

        return ModelTrainerConfig(
            train_data=config.train_data,
            test_data=config.test_data,
            model_dir=config.model_dir,
            model_ojb=config.model_obj,
            n_estimators=params.n_estimators,
            oob_score=params.oob_score)


## component

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, recall_score, accuracy_score, precision_score, classification_report

In [9]:
class ModelTrainer:
    def __init__(self,config:ModelTrainerConfig):
        self.config = config

    def initiate_model_training(self):
        config = self.config

        train_df = pd.read_csv(config.train_data)
        test_df = pd.read_csv(config.test_data)

        X_train = train_df.drop('Churn',axis=1)
        y_train = train_df['Churn']
        X_test = test_df.drop('Churn',axis=1)
        y_test = test_df['Churn']

        rfc = RandomForestClassifier(n_estimators=config.n_estimators,oob_score=config.oob_score)


        rfc.fit(X_train,y_train)
        logger.info(f'the {rfc} model trained successfully')
        joblib.dump(rfc,config.model_ojb)

        return rfc , X_test , y_test

    def evaluate(self,true,pred):
        
        cm = confusion_matrix(true, pred)
        accuracy = accuracy_score(true, pred)
        recall = recall_score(true, pred)
        
        precision = precision_score(true, pred)
        
        report = classification_report(true, pred)

        evaluation_report = {
            'confusion_matrix': cm,
            'accuracy': accuracy,
            'recall': recall,
            'precision': precision,
            'classification_report': report
        }
        logger.info(f'evaluation_report -> {evaluation_report}')
        
        return evaluation_report
    
    def train_model(self):
        model ,  X_test , y_test = self.initiate_model_training()

        # y_pred = model.predict(X_test)
        # self.evaluate(y_test,y_pred)
        

## pipeline

In [10]:
try:
    config = ConfigurationManager()
    trainer_config = config.get_modelTrainer_config()
    model_trainer = ModelTrainer(config=trainer_config)
    model_trainer.train_model()
except Exception as e:
    raise CustomException(e)


[2023-09-16 12:59:25,472: INFO: utils: yaml file: config\config.yaml loaded successfully]
[2023-09-16 12:59:25,477: INFO: utils: yaml file: schema.yaml loaded successfully]
[2023-09-16 12:59:25,480: INFO: utils: yaml file: params.yaml loaded successfully]
[2023-09-16 12:59:25,482: INFO: utils: Created artifacts]
[2023-09-16 12:59:25,484: INFO: utils: Created artifacts\model]


CustomException: Error occured in python script name [C:\Users\Gyanprakash\AppData\Local\Temp\ipykernel_5408\1690227895.py] line number [5] error message ["['Churn'] not found in axis"]